In [1]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.python.keras import backend as K
import matplotlib as plt
import math
from PIL import Image
import time

/home/harisankar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/harisankar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/harisankar/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/harisankar/.local/lib/python3.6/site-packages/tensorflow/python/framework

In [2]:
print(tf.keras.__version__)
print(tf.__version__)

2.2.4-tf
1.14.0


In [3]:
from imageai.Detection import ObjectDetection

Using TensorFlow backend.


In [4]:
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("yolo.h5")
detector.loadModel()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
file_name = "crowd.jpg"
out_file_name = file_name.split('.')[0]+"result.jpg"
print(out_file_name)

crowdresult.jpg


In [6]:
img = tf.keras.preprocessing.image.load_img(file_name)
img = tf.keras.preprocessing.image.img_to_array(img)
print(type(img))
img_shape = img.shape
img_height = img_shape[0]
img_width = img_shape[1]
print(img.shape)

<class 'numpy.ndarray'>
(540, 960, 3)


In [7]:
mid_h = math.ceil(img_height/2)
mid_w = math.ceil(img_width/2)
print(mid_h,mid_w)
offset_h = math.ceil(0.1*img_height)
offset_w = math.ceil(0.06*img_width)

# print(offset_h,offset_w)

p1 = (mid_h+offset_h)/img_height
p2 = (mid_w+offset_w)/img_width
p3 = (mid_h-offset_h)/img_height
p4 = (mid_w-offset_w)/img_width

# print(p1,p2)

box1 = [0,0,p1,p2]
box2 = [0,p3,p1,1]
box3 = [p4,0,1,p2]
box4 = [p4,p3,1,1]

box1 = np.array(box1)
box2 = np.array(box2)
box3 = np.array(box3)
box4 = np.array(box4)



270 480


In [8]:
draw_boxes = [[box1,box2,box3,box4]]
draw_boxes = np.array(draw_boxes)
print(draw_boxes)

[[[0.         0.         0.6        0.56041667]
  [0.         0.4        0.6        1.        ]
  [0.43958333 0.         1.         0.56041667]
  [0.43958333 0.4        1.         1.        ]]]


In [9]:
images = []
images.append(img)
images = np.array(images)
print(images.shape)

box1 = box1.reshape([4])
box2 = box2.reshape([4])
box3 = box3.reshape([4])
box4 = box4.reshape([4])

boxes = []
boxes.append(box1)
boxes.append(box2)
boxes.append(box3)
boxes.append(box4)
boxes = np.array(boxes)

print(boxes.shape)

crop_tensor = tf.image.crop_and_resize(
    images, boxes, [0,0,0,0], crop_size=[300,534], method='bilinear', extrapolation_value=0,
    name=None
)

cropped_images = crop_tensor.eval(session=tf.Session())

print(crop_tensor.shape)
print(type(cropped_images))
print(cropped_images.shape)

for i in range(4):
    result = tf.keras.preprocessing.image.array_to_img(cropped_images[i],data_format=None, scale=True, dtype=None)
    name = "abc"+str(i)+".jpg"
    result.save(name)

(1, 540, 960, 3)
(4, 4)
Instructions for updating:
box_ind is deprecated, use box_indices instead
(4, 300, 534, 3)
<class 'numpy.ndarray'>
(4, 300, 534, 3)


In [10]:
draw_tensor = tf.image.draw_bounding_boxes(images, draw_boxes, name=None) 
draw_img = draw_tensor.eval(session=tf.Session())
type(draw_img)
drawn_img = tf.keras.preprocessing.image.array_to_img(draw_img[0],data_format=None, scale=True, dtype=None)
drawn_img.save("boxes.jpg")

In [11]:
detections = detector.detectObjectsFromImage(input_image=file_name, output_image_path=out_file_name, minimum_percentage_probability=30,)

In [12]:
tic = time.time()
detections = []
for i in range(4):
    crop_det = detector.detectObjectsFromImage(input_image=cropped_images[i],input_type="array", output_image_path="crop_det"+str(i)+".jpg", minimum_percentage_probability=30)
    detections.append(crop_det)

tac = time.time()

elapsed = tac-tic
print(elapsed)

3.7826850414276123


In [13]:
detections[3]

[{'box_points': [489, 160, 502, 168],
  'name': 'sports ball',
  'percentage_probability': 43.53354573249817},
 {'box_points': [278, 27, 287, 52],
  'name': 'person',
  'percentage_probability': 39.784759283065796},
 {'box_points': [257, 27, 268, 44],
  'name': 'person',
  'percentage_probability': 42.853572964668274},
 {'box_points': [511, 7, 527, 39],
  'name': 'person',
  'percentage_probability': 42.89233982563019},
 {'box_points': [464, 3, 477, 36],
  'name': 'person',
  'percentage_probability': 46.661725640296936},
 {'box_points': [493, 12, 504, 36],
  'name': 'person',
  'percentage_probability': 48.408642411231995},
 {'box_points': [19, 282, 40, 300],
  'name': 'person',
  'percentage_probability': 54.931217432022095},
 {'box_points': [302, 86, 325, 117],
  'name': 'person',
  'percentage_probability': 61.790770292282104},
 {'box_points': [312, 45, 326, 80],
  'name': 'person',
  'percentage_probability': 62.81340718269348},
 {'box_points': [127, 112, 146, 152],
  'name': 'per